# Libs

In [ ]:
import yfinance as yf

In [ ]:
import pandas as pd

In [ ]:
!pip install torch

In [35]:
!pip install torch torch-geometric pandas scikit-learn

# Inital pull of data to see what it looks like

In [ ]:
stocks = ["AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "TSLA", "NVDA", "META", "UNH", "JNJ",
          "V", "JPM", "PG", "HD", "MA", "BAC", "XOM", "CVX", "LLY", "PFE"]

data = yf.download(stocks, period="1mo")
df = pd.DataFrame(data)
df.to_csv("stock_data.csv")

[*********************100%***********************]  20 of 20 completed


In [ ]:
df.head()

Price                       Adj Close                                     \
Ticker                           AAPL        AMZN        BAC         CVX   
Date                                                                       
2024-08-12 00:00:00+00:00  217.529999  166.800003  37.922756  143.414902   
2024-08-13 00:00:00+00:00  221.270004  170.229996  38.210880  142.554520   
2024-08-14 00:00:00+00:00  221.720001  170.100006  38.558617  143.068771   
2024-08-15 00:00:00+00:00  224.720001  177.589996  38.777187  145.175201   
2024-08-16 00:00:00+00:00  226.050003  177.059998  39.085182  145.639999   

Price                                                                      \
Ticker                           GOOG       GOOGL          HD         JNJ   
Date                                                                        
2024-08-12 00:00:00+00:00  163.734467  162.074921  343.722290  158.675629   
2024-08-13 00:00:00+00:00  165.711853  163.942459  347.956573  157.196854   
2024-08-14 00:00:00+00:00  161.816986  160.157471  353.512817  157.286179   
2024-08-15 00:00:00+00:00  162.955490  161.086243  357.896210  157.891586   
2024-08-16 00:00:00+00:00  164.523438  162.744049  359.874176  158.189316   

Price                                              ...   Volume            \
Ticker                            JPM         LLY  ...       MA      META   
Date                                               ...                      
2024-08-12 00:00:00+00:00  206.190002  883.145996  ...  1760000   9767400   
2024-08-13 00:00:00+00:00  207.940002  906.782959  ...  1816100  13743800   
2024-08-14 00:00:00+00:00  210.240005  930.380005  ...  1654100  11444200   
2024-08-15 00:00:00+00:00  211.550003  931.580017  ...  2130500  13482100   
2024-08-16 00:00:00+00:00  213.970001  922.119995  ...  1921900  14776700   

Price                                                                        \
Ticker                         MSFT       NVDA       PFE       PG      TSLA   
Date                                                                          
2024-08-12 00:00:00+00:00  16762900  325559900  21141600  7208100  64044900   
2024-08-13 00:00:00+00:00  19414300  312646700  34620700  3770000  76247400   
2024-08-14 00:00:00+00:00  18267000  339246400  23677700  5476600  70250000   
2024-08-15 00:00:00+00:00  20752100  318086700  28676800  9491400  89848500   
2024-08-16 00:00:00+00:00  22775600  302589900  50332900  5640800  88765100   

Price                                                   
Ticker                         UNH         V       XOM  
Date                                                    
2024-08-12 00:00:00+00:00  2532400  26769400  13228900  
2024-08-13 00:00:00+00:00  2369400  12005700  11585100  
2024-08-14 00:00:00+00:00  2344400   8552300  12194500  
2024-08-15 00:00:00+00:00  2751800   7397800  11579400  
2024-08-16 00:00:00+00:00  2728900   5111100  10056000  

[5 rows x 120 columns]

# Fine tuned data API call

just pulling the stock data would only give us the prices at the end of that, that is not what we would be looking at for our project, we would want some more data that we could base our model off.

In [60]:
info = ['sector', 'industry', 'fullTimeEmployees', 'profitMargins',
        'operatingMargins', 'returnOnAssets', 'returnOnEquity',
        'revenueGrowth', 'earningsGrowth', 'debtToEquity', 'totalCash',
        'totalDebt', 'totalRevenue', 'bookValue', 'operatingCashflow',
        'freeCashflow', 'targetLowPrice', 'targetMeanPrice',
        'targetMedianPrice', 'recommendationMean']

data = []
stocks = ["AAPL", "MSFT", "AMZN", "GOOG", "GOOGL", "TSLA", "NVDA", "META", "UNH", "JNJ", "V", "JPM", "PG", "HD", "MA", "BAC", "XOM", "CVX", "LLY", "PFE"]

# Download closing prices
prices = yf.download(stocks, period="1d")['Close']

for stock in stocks:
  ticker = yf.Ticker(stock)
  stock_info = [stock]
  for i in info:
    try:
      stock_info.append(ticker.info[i])
    except KeyError:
      stock_info.append(None)
  # Add closing price to stock_info
  stock_info.append(prices[stock].iloc[0])
  data.append(stock_info)

df = pd.DataFrame(data, columns = ['Ticker'] + info + ['Close'])

[*********************100%***********************]  20 of 20 completed


In [61]:
df.head()

,Ticker,sector,industry,fullTimeEmployees,profitMargins,operatingMargins,returnOnAssets,returnOnEquity,revenueGrowth,earningsGrowth,...,totalDebt,totalRevenue,bookValue,operatingCashflow,freeCashflow,targetLowPrice,targetMeanPrice,targetMedianPrice,recommendationMean,Close
0,AAPL,Technology,Consumer Electronics,161000.0,0.26441,0.29556,0.22612,1.60583,0.049,0.111,...,101304000512,385603010560,4.382,113040998400,8.615812e+10,183.86,240.58,243.0,2.0,220.110001
1,MSFT,Technology,Software - Infrastructure,228000.0,0.35956,0.43143,0.14802,0.37133,0.152,0.097,...,97851998208,245122007040,36.115,118547996672,5.670525e+10,440.00,496.67,500.0,1.7,414.200012
2,AMZN,Consumer Cyclical,Internet Retail,1525000.0,0.07350,0.09915,0.06584,0.21933,0.101,0.938,...,157841997824,604333998080,22.540,107951996928,5.919100e+10,180.00,218.35,220.0,1.8,179.550003
3,GOOG,Communication Services,Internet Content & Information,179582.0,0.26702,0.32363,0.15961,0.30871,0.136,0.314,...,28718999552,328284012544,24.408,105059000320,4.398850e+10,145.00,200.58,204.5,1.8,150.009995
4,GOOGL,Communication Services,Internet Content & Information,179582.0,0.26702,0.32363,0.15961,0.30871,0.136,0.314,...,28718999552,328284012544,24.408,105059000320,4.398850e+10,170.00,202.66,202.0,1.9,148.660004


In [62]:
df['Close']

,Close
0,220.110001
1,414.200012
2,179.550003
3,150.009995
4,148.660004
5,226.169998
6,108.099998
7,504.790009
8,599.030029
9,167.380005


In [63]:
df.to_csv("stock_data.csv")

# Modeling

In [40]:
import torch
from torch_geometric.data import Data
from sklearn.preprocessing import StandardScaler
import pandas as pd

In [64]:
print(df.isna().sum())

Ticker                0
sector                0
industry              0
fullTimeEmployees     1
profitMargins         0
operatingMargins      0
returnOnAssets        0
returnOnEquity        0
revenueGrowth         0
earningsGrowth        0
debtToEquity          2
totalCash             0
totalDebt             0
totalRevenue          0
bookValue             0
operatingCashflow     0
freeCashflow          2
targetLowPrice        0
targetMeanPrice       0
targetMedianPrice     0
recommendationMean    0
Close                 0
dtype: int64


In [66]:
df_cleaned = df.dropna()

In [83]:
df_cleaned = df.dropna(how='any').reset_index(drop=True)

In [68]:
df_cleaned.isna().sum()

,0
Ticker,0
sector,0
industry,0
fullTimeEmployees,0
profitMargins,0
operatingMargins,0
returnOnAssets,0
returnOnEquity,0
revenueGrowth,0
earningsGrowth,0


In [90]:
from sklearn.preprocessing import StandardScaler


df_encoded = pd.get_dummies(df_cleaned.drop(columns=['Ticker']), columns=['sector', 'industry'], drop_first=True)
df_encoded = df_encoded.astype(int)

features = ['fullTimeEmployees', 'profitMargins', 'operatingMargins', 'returnOnAssets',
            'returnOnEquity', 'revenueGrowth', 'earningsGrowth', 'debtToEquity',
            'totalCash', 'totalDebt', 'totalRevenue', 'bookValue',
            'operatingCashflow', 'freeCashflow', 'targetLowPrice',
            'targetMeanPrice', 'targetMedianPrice', 'recommendationMean']

scaler = StandardScaler()
df_encoded[features] = scaler.fit_transform(df_encoded[features])



In [85]:
df_encoded.head()

,fullTimeEmployees,profitMargins,operatingMargins,returnOnAssets,returnOnEquity,revenueGrowth,earningsGrowth,debtToEquity,totalCash,totalDebt,...,industry_Credit Services,industry_Drug Manufacturers - General,industry_Healthcare Plans,industry_Home Improvement Retail,industry_Household & Personal Products,industry_Internet Content & Information,industry_Internet Retail,industry_Oil & Gas Integrated,industry_Semiconductors,industry_Software - Infrastructure
0,-0.202730,0.251772,0.094137,0.683468,0.687810,-0.418041,-0.127195,0.006303,0.663202,1.316287,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.009328,0.843014,0.932657,0.027158,-0.331532,-0.048349,-0.151432,-0.334637,1.067419,1.225477,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,3.734593,-0.934503,-1.118005,-0.663438,-0.457041,-0.231401,1.304539,-0.245103,1.466660,2.803591,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,-0.149092,0.267990,0.267371,0.124554,-0.383239,-0.105777,0.224246,-0.414095,1.809141,-0.593155,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,-0.149092,0.267990,0.267371,0.124554,-0.383239,-0.105777,0.224246,-0.414095,1.809141,-0.593155,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [91]:
X = df_encoded.drop(columns=['Close']).values
y = df_encoded['Close'].values

# Convert to torch tensors
x = torch.tensor(X, dtype=torch.float)
y = torch.tensor(y, dtype=torch.float)

In [92]:
edge_index = []

for i in range(len(df_cleaned)):
    for j in range(i + 1, len(df_cleaned)):
        if df_cleaned.iloc[i]['sector'] == df.iloc[j]['sector']:
            edge_index.append([i, j])
            edge_index.append([j, i])

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

data = Data(x=x, edge_index=edge_index, y=y)

In [93]:
import torch.nn as nn
from torch_geometric.nn import GCNConv

class GNN(torch.nn.Module):
    def __init__(self):
        super(GNN, self).__init__()
        self.conv1 = GCNConv(in_channels=x.shape[1], out_channels=64)
        self.conv2 = GCNConv(in_channels=64, out_channels=32)
        self.fc = nn.Linear(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        x = torch.relu(x)
        x = self.fc(x)
        return x

model = GNN()

In [94]:
import torch.optim as optim


optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    output = model(data)
    loss = criterion(output.flatten(), data.y)
    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

Epoch 0, Loss: 132265.265625
Epoch 10, Loss: 132137.328125
Epoch 20, Loss: 131972.0
Epoch 30, Loss: 131723.375
Epoch 40, Loss: 131350.890625
Epoch 50, Loss: 130786.4375
Epoch 60, Loss: 129946.6484375
Epoch 70, Loss: 128737.9140625
Epoch 80, Loss: 127053.6875
Epoch 90, Loss: 124771.03125
Epoch 100, Loss: 121785.4609375
Epoch 110, Loss: 118021.21875
Epoch 120, Loss: 113418.3203125
Epoch 130, Loss: 107943.546875
Epoch 140, Loss: 101608.25
Epoch 150, Loss: 94479.484375
Epoch 160, Loss: 86699.8984375
Epoch 170, Loss: 78512.46875
Epoch 180, Loss: 70229.296875
Epoch 190, Loss: 62217.91015625


While the results were not super accurate as of yet, there is alot of room to grow here and build off this. This is a simple config of this idea and there was not alot of fine tuning done yet, there is alot more information that can be used to bring those numbers much closer to each other. This is a good base of ideas from other projects along with my research of many different papers for our group to use moving forward and build off this idea


In [98]:
model.eval()
with torch.no_grad():
    predictions = model(data).flatten().detach().numpy()

actual_prices = y.detach().numpy()
tickers = df_cleaned['Ticker'].values

comparison_df = pd.DataFrame({'Ticker': tickers, 'Actual Price': actual_prices, 'Predicted Price': predictions})
print(comparison_df)

   Ticker  Actual Price  Predicted Price
0    AAPL         220.0       162.689148
1    MSFT         414.0       162.689148
2    AMZN         179.0       102.632736
3    GOOG         150.0       135.537048
4   GOOGL         148.0       135.537048
5    TSLA         226.0       102.632736
6    NVDA         108.0       162.689148
7    META         504.0       135.537048
8     UNH         599.0       192.828903
9     JNJ         167.0       192.828903
10     PG         177.0       189.076187
11     HD         370.0       106.900169
12     MA         488.0       285.328949
13    XOM         110.0       155.114975
14    CVX         138.0       182.940765
15    LLY         901.0       256.052582
16    PFE          29.0       127.060226


In [97]:
results_df = pd.DataFrame({'Ticker': df_cleaned['Ticker'], 'Predicted Price': predictions})
print(results_df)

   Ticker  Predicted Price
0    AAPL       162.689148
1    MSFT       162.689148
2    AMZN       102.632736
3    GOOG       135.537048
4   GOOGL       135.537048
5    TSLA       102.632736
6    NVDA       162.689148
7    META       135.537048
8     UNH       192.828903
9     JNJ       192.828903
10     PG       189.076187
11     HD       106.900169
12     MA       285.328949
13    XOM       155.114975
14    CVX       182.940765
15    LLY       256.052582
16    PFE       127.060226


# Trouble shooting

In [57]:
np.isinf(df_encoded).sum()

,0
Unnamed: 0,0
fullTimeEmployees,0
profitMargins,0
operatingMargins,0
returnOnAssets,0
returnOnEquity,0
revenueGrowth,0
earningsGrowth,0
debtToEquity,0
totalCash,0


In [58]:
print(torch.isnan(x).sum())
print(torch.isinf(x).sum())
print(torch.isnan(y).sum())
print(torch.isinf(y).sum())

tensor(5)
tensor(0)
tensor(0)
tensor(0)


In [59]:
print(output)

tensor([[nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan],
        [nan]], grad_fn=<AddmmBackward0>)
